<a href="https://colab.research.google.com/github/Captmoonshot/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/Sammy%20Lee%20-%20Gradient_Descent_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent Implementation Challenge!!

## Use gradient descent to find the optimal parameters of a **multiple** regression model. (We only showed an implementation for a bivariate model during lecture.)

A note: Implementing gradient descent in any context is not trivial, particularly the step where we calculate the gradient will change based on the number of parameters that we're trying to optimize for. You will need to research what the gradient of a multiple regression model looks like. This challenge is pretty open-ended but I hope it will be thrilling. Please work together, help each other, share resources and generally expand your understanding of gradient descent as you try and achieve this implementation. 

## Suggestions:

Start off with a model that has just two $X$ variables You can use any datasets that have at least two x variables. Potential candidates might be the blood pressure dataset that we used during lecture on Monday: [HERE](https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls) or any of the housing datasets. You would just need to select from them the two varaibles $x$ variables and one y variable that you want to work with that you most want to work with. 

Use Sklearn to find the optimal parameters of your model first. (like we did during the lecture.) So that you can compare the parameter estimates of your gradient-descent linear regression to the estimates of OLS linear regression. If implemented correctly they should be nearly identical.

Becoming a Data Scientist is all about striking out into the unknown, getting stuck and then researching and fighting and learning until you get yourself unstuck. Work together! And fight to take your own learning-rate fueled step towards your own optimal understanding of gradient descent! 


### Using Boston Housing DataSet

In [0]:
##### Make it Hap'n Cap'n #####

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [96]:
df = pd.read_csv('https://raw.githubusercontent.com/rasbt/'
                 'python-machine-learning-book-2nd-edition'
                 '/master/code/ch10/housing.data.txt',
                header=None,
                sep='\s+')

df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 
              'NOX', 'RM', 'AGE', 'DIS', 'RAD', 
              'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### Using Scikit-Learn as our Baseline

In [97]:
# Gradient Descent DataFrame

GD = df[['CRIM', 'RM', 'MEDV']]
GD.head()

,CRIM,RM,MEDV
0,0.00632,6.575,24.0
1,0.02731,6.421,21.6
2,0.02729,7.185,34.7
3,0.03237,6.998,33.4
4,0.06905,7.147,36.2


In [98]:
GD.shape

(506, 3)

In [99]:
X = GD.loc[:, 'CRIM':'RM'].values
y = GD['MEDV'].values

print(X.shape)
print(y.shape)

(506, 2)
(506,)


In [100]:
# standardize X

X = (X - X.mean()) / X.std()

X[:5]

array([[-0.79198032,  0.260522  ],
       [-0.78861708,  0.23584652],
       [-0.78862028,  0.35826255],
       [-0.78780631,  0.32829946],
       [-0.78192906,  0.3521738 ]])

In [101]:
# Baseline model to compare Gradient Descent alogrithm with

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

beta_1 = model.coef_
beta_0 = model.intercept_

print("intercept: {}".format(beta_0))
print("beta coefficients: {}".format(beta_1))
print("Training Score: {}".format(model.score(X, y)))

intercept: 10.97226336336069
beta coefficients: [-1.65332688 52.36876093]
Training Score: 0.5419591738494083


## Let's Start on our Gradient Descent Algorithm

In [102]:
crime = X[:, 0]
room = X[:, 1]
price = df['MEDV'].values

m = len(crime)
x0 = np.ones(m)
X = np.array([x0, crime, room]).T

X[:5]

array([[ 1.        , -0.79198032,  0.260522  ],
       [ 1.        , -0.78861708,  0.23584652],
       [ 1.        , -0.78862028,  0.35826255],
       [ 1.        , -0.78780631,  0.32829946],
       [ 1.        , -0.78192906,  0.3521738 ]])

In [0]:
# Initial Coefficients
B = np.array([0, 0, 0]) # theta values
y = np.array(price)
alpha = 0.001

In [0]:
# cost_function MSE

def cost_function(X, y, B):
    m = len(y)
    J = np.sum((X.dot(B) - y) ** 2) / (2 * m)
    return J

In [106]:
# Our initial MSE is obviously huge

initial_cost = cost_function(X, y, B)

print("Gradient Descent Initial Cost:", initial_cost)

Gradient Descent Initial Cost: 296.0734584980237


In [0]:
def gradient_descent(X, y, B, alpha, iterations):
    cost_history = [0] * iterations
    m = len(y)
    
    for iteration in range(iterations):
        # Hypothesis Values
        h = X.dot(B)
        # Difference b/w Hypothesis and Actual Y
        loss = h - y
        # Gradient Calculation
        gradient = X.T.dot(loss) / m
        # Changing Values of B using Gradient
        B = B - alpha * gradient
        # New Cost Value
        cost = cost_function(X, y, B)
        cost_history[iteration] = cost
    
    return B, cost_history

In [109]:
# Compute the final value of B
# 100000
newB, cost_history = gradient_descent(X, y, B, 0.01, 1000000)

# new values of B
print(newB)

# Final Cost of new B
print(cost_history[-1])

[10.97226336 -1.65332688 52.36876093]
19.33380162251817


## The above coefficients and intercept match the results from scikit-learn above:
intercept: 10.97226336336069
beta coefficients: [-1.65332688 52.36876093]

## and the final cost is much smaller than our initial MSE going from 296 to 19

## Stretch Goals

If you happen upon the most useful resources for accomplishing this challenge first, I want you to spend time today studying other variations of Gradient Descent-Based Optimizers.

- Try and write a function that can perform gradient descent for arbitarily large (in dimensionality) multiple regression models. 
- Create a notebook for yourself exploring these topics
- How do they differ from the "vanilla" gradient descent we explored today
- How do these different gradient descent-based optimizers seek to overcome the challenge of finding the global minimum among various local minima?
- Write a blog post that reteaches what you have learned about these other gradient descent-based optimizers.

[Overview of GD-based optimizers](http://ruder.io/optimizing-gradient-descent/)

[Siraj Raval - Evolution of Gradient Descent-Based Optimizers](https://youtu.be/nhqo0u1a6fw)